# Three Methods of Prompt Configuration

This notebook demonstrates different approaches to configuring agent prompts.

## Key Concepts
- **Method 1**: String Prompt (simplest)
- **Method 2**: SystemMessage (structured)

## When to Use Each Method
- **String**: Simple, static agents
- **SystemMessage**: Production chat agents

In [1]:
# Import required modules
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
from typing import TypedDict
import tools

## Method 1: Simple String Prompt

The simplest approach - direct string instruction to the agent.

In [2]:
print("=== Method 1: Simple String Prompt ===")

# Create model instance
model = ChatOllama(model="qwen3")

# Method 1: String prompt (simplest)
agent1 = create_agent(
    model,
    [tools.helper_tool],
    prompt="You are a helpful assistant. Be concise and accurate in your responses."
)

print("✓ Agent 1 created with string prompt")
print("  Characteristics:")
print("  - Direct string instruction")
print("  - Simple and readable")
print("  - Good for basic use cases")
print("  - Easy to modify")

=== Method 1: Simple String Prompt ===
✓ Agent 1 created with string prompt
  Characteristics:
  - Direct string instruction
  - Simple and readable
  - Good for basic use cases
  - Easy to modify


## Method 2: SystemMessage Prompt

Using LangChain's SystemMessage class for more structured prompt handling.

In [3]:
print("=== Method 2: SystemMessage Prompt ===")

# Method 2: SystemMessage prompt (structured)
agent2 = create_agent(
    model,
    [tools.helper_tool],
    prompt=SystemMessage(content="You are a research assistant. Always cite your sources and provide detailed explanations.")
)

print("✓ Agent 2 created with SystemMessage prompt")
print("  Characteristics:")
print("  - Structured message object")
print("  - Better integration with chat models")
print("  - More explicit about message type")
print("  - Professional for production use")

=== Method 2: SystemMessage Prompt ===
✓ Agent 2 created with SystemMessage prompt
  Characteristics:
  - Structured message object
  - Better integration with chat models
  - More explicit about message type
  - Professional for production use


## Testing All Two Types of Prompt

Let's test all agents with the same question to see their different behaviors:

In [4]:
# Test question for all agents
test_question = "Help me understand artificial intelligence"

print(f"Testing all agents with: '{test_question}'")
print("=" * 70)

# Test Agent 1 (String prompt)
print("\n=== Agent 1 Response (String Prompt) ===")
print("Expected: Concise and accurate response")

result1 = agent1.invoke({"messages": test_question})
print(f"Response: {result1['messages'][-1].content}")

# Test Agent 2 (SystemMessage prompt)
print("\n=== Agent 2 Response (SystemMessage Prompt) ===")
print("Expected: Detailed explanation with research focus")

result2 = agent2.invoke({"messages": test_question})
print(f"Response: {result2['messages'][-1].content}")

Testing all agents with: 'Help me understand artificial intelligence'

=== Agent 1 Response (String Prompt) ===
Expected: Concise and accurate response
Response: <think>
Okay, the user is asking for help understanding artificial intelligence. They've already received a general explanation, but they're responding with a tool call, which might indicate they want a more structured or detailed breakdown. Let me check the tools available. The helper_tool is designed for various assistance requests, so I should use that to provide a comprehensive explanation. I'll need to cover the definition, key concepts, applications, and implications of AI. I'll structure this in a clear and concise manner, avoiding jargon. Finally, I'll ask if they need more details on any specific area. Let me make sure the response is helpful and easy to follow.
</think>

**Artificial Intelligence (AI)** is a branch of computer science focused on creating systems that can perform tasks requiring human intelligence, su